In [23]:
import os

if not os.path.exists("my_new_directory"):
    os.mkdir("my_new_directory")
else:
    print("Directory 'my_new_directory' already exists.")

Directory 'my_new_directory' already exists.


In [24]:
import os
import pandas as pd
import json
import matplotlib.pyplot as plt

import folium
from folium.plugins import HeatMap
dir_path = "my_new_directory" # Using the directory created in the previous cell
files = os.listdir(dir_path)
print("Total no. of files:", len(files))

Total no. of files: 10


In [26]:
with open(os.path.join(dir_path, files[0])) as f:
    json_object = json.load(f)

print(len(json_object))

keys = list(json_object[0].keys())
print("The keys in data are:", keys)

df_main = pd.DataFrame(data = {}, columns = keys)
df_main.columns[df_main.columns.duplicated()].to_list()

for i in range(len(files)):
  with open(os.path.join(dir_path, files[i])) as f:
    json_obj = json.load(f)
    df_main = pd.concat([df_main, pd.DataFrame(json_obj)], ignore_index=True)

print(df_main.shape)
df = df_main.copy()
df.head()



27217
The keys in data are: ['full_text', 'text_translation', 'created_at', 'screen_name', 'description', 'desc_translation', 'weekofyear', 'weekday', 'day', 'month', 'year', 'location', 'point_info', 'point', 'latitude', 'longitude', 'altitude', 'province', 'hisco_standard', 'hisco_code', 'industry', 'sentiment_pattern', 'subjective_pattern']


/tmp/ipython-input-26-3013524848.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_main = pd.concat([df_main, pd.DataFrame(json_obj)], ignore_index=True)


(271342, 23)


,full_text,text_translation,created_at,screen_name,description,desc_translation,weekofyear,weekday,day,month,...,point,latitude,longitude,altitude,province,hisco_standard,hisco_code,industry,sentiment_pattern,subjective_pattern
0,"RT @chrisilluminati: Much like the pandemic, t...","RT @chrisilluminati: Much like the pandemic, t...",1593530905000,hejost67,"Sports, nature, climate, food...","Sports, nature, climate, food ...",27.0,1.0,30.0,6.0,...,"(52.5001698, 5.7480821, 0.0)",52.500170,5.748082,0.0,Flevoland,min,7410,True,0.0000,0.000
1,RT @Dirk_Lauwers: Visie van collega @KBoussauw...,RT @Dirk_Lauwers: Vision @KBoussauw colleagues...,1593672474000,KarelBrits,"If people are not laughing at your goals, your...","If people are not laughing at your goals, your...",27.0,3.0,2.0,7.0,...,"(51.0312293, 4.098112, 0.0)",51.031229,4.098112,0.0,False,None,None,False,0.3000,0.500
2,夏は、甘いのが人気！\n #stay home\nブ\n ラ\n ッ\n ク...,"Summer, sweet popular!\n#stay home\nBreakfast\...",1593672475000,BlackDe74164200,Black Devil 👿再び降臨‼️ 「BLACK DEVIL（ブラックデビル）」は、🇳🇱...,"Black Devil 👿 again descended! ️ ""BLACK DEVIL ...",27.0,3.0,2.0,7.0,...,"(52.5001698, 5.7480821, 0.0)",52.500170,5.748082,0.0,Flevoland,None,None,False,0.0000,0.000
3,"Corona, het mooiste excuus om ons elke nog tas...","Corona, the best excuse to take us any more ta...",1593530908000,Chasidiyah,Je weet nooit wat je voor n ander betekent. Li...,You never know what you mean to one another. I...,27.0,1.0,30.0,6.0,...,None,NaN,NaN,NaN,False,dienst,55220,True,0.0625,0.275
4,@mevrouwadvocaat De Chinesetomatensoep wordt d...,@mevrouwadvocaat Chinese Tomato caught twice. ...,1581174053000,Glijn1,Zoon van de ouwe Glijn en trotse vader van Gli...,Son of the old Glijn and proud father of Glijn...,6.0,5.0,8.0,2.0,...,None,NaN,NaN,0.0,False,advocaat,12110,True,0.0000,0.000


In [29]:
df_map = df[['created_at', 'location', 'latitude', 'longitude', 'altitude', 'province']]
df_map.head()
print(df_map.shape)

(271342, 6)


In [30]:
df_map.dropna(subset=['longitude', 'latitude'], inplace=True)

/tmp/ipython-input-30-2587569966.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_map.dropna(subset=['longitude', 'latitude'], inplace=True)


In [31]:
df_map = df_map[(df_map['latitude'] >= -90) & (df_map['latitude'] <= 90)]
df_map = df_map[(df_map['longitude'] >= -180) & (df_map['longitude'] <= 180)]

In [32]:
df_map['latitude'] = df_map['latitude'].round(4)
df_map['longitude'] = df_map['longitude'].round(4)

In [33]:
df_map.shape

(134445, 6)

In [34]:
df_group = df_map.drop(['location' ,'altitude','province'], axis='columns')

In [35]:
map_center = [df_group['longitude'].mean(), df_group['latitude'].mean()]
heat_map = folium.Map(locations = map_center, zoom_start=4)

In [36]:
head_data = df_group[['latitude', 'longitude']].values.tolist()
HeatMap(head_data, radius=10, blur=15, max_zoom=6).add_to(heat_map)

In [37]:
image = "dutch_heatmap.html"
heat_map.save(image)
print(df['full_text'].head())
print(df['text_translation'].head())

0    RT @chrisilluminati: Much like the pandemic, t...
1    RT @Dirk_Lauwers: Visie van collega @KBoussauw...
2    夏は、甘いのが人気！\n　　　　#stay home\nブ\n　 ラ\n　 ッ\n　 　 ク...
3    Corona, het mooiste excuus om ons elke nog tas...
4    @mevrouwadvocaat De Chinesetomatensoep wordt d...
Name: full_text, dtype: object
0    RT @chrisilluminati: Much like the pandemic, t...
1    RT @Dirk_Lauwers: Vision @KBoussauw colleagues...
2    Summer, sweet popular!\n#stay home\nBreakfast\...
3    Corona, the best excuse to take us any more ta...
4    @mevrouwadvocaat Chinese Tomato caught twice. ...
Name: text_translation, dtype: object


In [38]:
df_comments = df[['text_translation']]
import nltk

nltk.download([
"names",
"stopwords",
"state_union",
"twitter_samples",
"movie_reviews",
"averaged_perceptron_tagger",
"vader_lexicon",
"punkt",
])

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to /root/nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   

True

In [39]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
df_comments.head()
df_comments.dropna(inplace=True)

In [40]:
def senti_analysis(text):
    scores = sia.polarity_scores(text)
    return scores['compound']
df_comments['sentiment_scores'] = df_comments['text_translation'].apply(senti_analysis)

/tmp/ipython-input-40-2208162249.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comments['sentiment_scores'] = df_comments['text_translation'].apply(senti_analysis)


In [41]:
def sentiments(scores):
    if scores > 0.05:
        return "positive"

    elif scores <= -0.05:
        return "negative"

    else:
        return "neutral"
df_comments['sentiment_label'] = df_comments['sentiment_scores'].apply(sentiments)

df_comments.to_csv("sentimentAnalysis.csv", index = False)

/tmp/ipython-input-41-1214548404.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comments['sentiment_label'] = df_comments['sentiment_scores'].apply(sentiments)
